In [1]:
from collections import Counter

users_interests = [
    ["Hadoop", "Big Data", "HBase", "Java", "Spark", "Storm", "Cassandra"],
    ["NoSQL", "MongoDB", "Cassandra", "HBase", "Postgres"],
    ["Python", "scikit-learn", "scipy", "numpy", "statsmodels", "pandas"],
    ["R", "Python", "statistics", "regression", "probability"],
    ["machine learning", "regression", "decision trees", "libsvm"],
    ["Python", "R", "Java", "C++", "Haskell", "programming languages"],
    ["statistics", "probability", "mathematics", "theory"],
    ["machine learning", "scikit-learn", "Mahout", "neural networks"],
    ["neural networks", "deep learning", "Big Data", "artificial intelligence"],
    ["Hadoop", "Java", "MapReduce", "Big Data"],
    ["statistics", "R", "statsmodels"],
    ["C++", "deep learning", "artificial intelligence", "probability"],
    ["pandas", "R", "Python"],
    ["databases", "HBase", "Postgres", "MySQL", "MongoDB"],
    ["libsvm", "regression", "support vector machines"]
]

popular_interests = Counter(
    [interest for user_interests in users_interests
                                for interest in user_interests]
).most_common()
popular_interests

[('Python', 4),
 ('R', 4),
 ('Big Data', 3),
 ('HBase', 3),
 ('Java', 3),
 ('statistics', 3),
 ('regression', 3),
 ('probability', 3),
 ('Hadoop', 2),
 ('Cassandra', 2),
 ('MongoDB', 2),
 ('Postgres', 2),
 ('scikit-learn', 2),
 ('statsmodels', 2),
 ('pandas', 2),
 ('machine learning', 2),
 ('libsvm', 2),
 ('C++', 2),
 ('neural networks', 2),
 ('deep learning', 2),
 ('artificial intelligence', 2),
 ('Spark', 1),
 ('Storm', 1),
 ('NoSQL', 1),
 ('scipy', 1),
 ('numpy', 1),
 ('decision trees', 1),
 ('Haskell', 1),
 ('programming languages', 1),
 ('mathematics', 1),
 ('theory', 1),
 ('Mahout', 1),
 ('MapReduce', 1),
 ('databases', 1),
 ('MySQL', 1),
 ('support vector machines', 1)]

In [2]:
def most_popular_new_interests(user_interests, max_results = 5):
    suggestions = [(interest, freq) for interest, freq in popular_interests if interest not in user_interests]
    return suggestions[:max_results]


most_popular_new_interests(users_interests[9])

[('Python', 4), ('R', 4), ('HBase', 3), ('statistics', 3), ('regression', 3)]

## User-Based Collaborative Filtering

In [3]:
import math
import numpy as np

def cosine_similarity(v, w):
    return np.dot(v, w) / math.sqrt(np.dot(v, v) * np.dot(w, w))

unique_interests = set([interest for user_interests in users_interests 
                                for interest in user_interests])
unique_interests = sorted(list(unique_interests))
unique_interests

['Big Data',
 'C++',
 'Cassandra',
 'HBase',
 'Hadoop',
 'Haskell',
 'Java',
 'Mahout',
 'MapReduce',
 'MongoDB',
 'MySQL',
 'NoSQL',
 'Postgres',
 'Python',
 'R',
 'Spark',
 'Storm',
 'artificial intelligence',
 'databases',
 'decision trees',
 'deep learning',
 'libsvm',
 'machine learning',
 'mathematics',
 'neural networks',
 'numpy',
 'pandas',
 'probability',
 'programming languages',
 'regression',
 'scikit-learn',
 'scipy',
 'statistics',
 'statsmodels',
 'support vector machines',
 'theory']

In [4]:
def make_user_interest_vector(users_interests):
    return [1 if interest in users_interests else 0 for interest in unique_interests]

user_interest_matrix = list(map(make_user_interest_vector, users_interests))

user_similarities = [[cosine_similarity(interest_vector_i, interest_vector_j)
                      for interest_vector_j in user_interest_matrix]
                     for interest_vector_i in user_interest_matrix]

user_similarities

[[1.0,
  0.3380617018914066,
  0.0,
  0.0,
  0.0,
  0.1543033499620919,
  0.0,
  0.0,
  0.1889822365046136,
  0.5669467095138409,
  0.0,
  0.0,
  0.0,
  0.1690308509457033,
  0.0],
 [0.3380617018914066,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.6,
  0.0],
 [0.0,
  0.0,
  1.0,
  0.18257418583505536,
  0.0,
  0.16666666666666666,
  0.0,
  0.20412414523193154,
  0.0,
  0.0,
  0.23570226039551587,
  0.0,
  0.47140452079103173,
  0.0,
  0.0],
 [0.0,
  0.0,
  0.18257418583505536,
  1.0,
  0.22360679774997896,
  0.3651483716701107,
  0.4472135954999579,
  0.0,
  0.0,
  0.0,
  0.5163977794943222,
  0.22360679774997896,
  0.5163977794943222,
  0.0,
  0.2581988897471611],
 [0.0,
  0.0,
  0.0,
  0.22360679774997896,
  1.0,
  0.0,
  0.0,
  0.25,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.5773502691896258],
 [0.1543033499620919,
  0.0,
  0.16666666666666666,
  0.3651483716701107,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.20412414523193154,
  0.235702260

In [5]:
def most_similar_users_to(user_id):
    result = [(id, simi) 
              for (id, simi) in enumerate(user_similarities[user_id])
              if simi > 0 and id != user_id]
    return sorted(result, key=lambda x: x[1], reverse=True)

most_similar_users_to(0)

[(9, 0.5669467095138409),
 (1, 0.3380617018914066),
 (8, 0.1889822365046136),
 (13, 0.1690308509457033),
 (5, 0.1543033499620919)]

In [6]:
from collections import defaultdict

def user_based_suggestions(user_id, include_current_interests=False):
    # sum up the similarities
    suggestions = defaultdict(float)
    for other_user_id, similarity in most_similar_users_to(user_id):
        for interest in users_interests[other_user_id]:
            suggestions[interest] += similarity
    
    # convert them to a sorted list
    suggestions = sorted(suggestions.items(), 
                         key=lambda x: x[1], reverse=True)
    # exclude already-interests
    if include_current_interests:
        return suggestions
    else:
        return [(suggestion, weight)
                 for suggestion, weight in suggestions
                 if suggestion not in users_interests[user_id]]
    
    

In [7]:
user_based_suggestions(0)

[('MapReduce', 0.5669467095138409),
 ('MongoDB', 0.50709255283711),
 ('Postgres', 0.50709255283711),
 ('NoSQL', 0.3380617018914066),
 ('neural networks', 0.1889822365046136),
 ('deep learning', 0.1889822365046136),
 ('artificial intelligence', 0.1889822365046136),
 ('databases', 0.1690308509457033),
 ('MySQL', 0.1690308509457033),
 ('Python', 0.1543033499620919),
 ('R', 0.1543033499620919),
 ('C++', 0.1543033499620919),
 ('Haskell', 0.1543033499620919),
 ('programming languages', 0.1543033499620919)]

Lưu ý nếu như trả về cả những exist interest của user thì về phần weight sẽ k có ý nghĩa lắm vì có thể exist interest của user sẽ có weight thấp hơn các interest còn lại

## Item-Based Collaborative Filtering

In [8]:
interst_user_matrix = [
    [user_interest_vector[j]   
    for user_interest_vector in user_interest_matrix]
    for j, _ in enumerate(unique_interests)]

print(interst_user_matrix)

[[1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0], [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0,

In [13]:
interest_similarities = [[cosine_similarity(user_vector_i, user_vector_j) for user_vector_j in interst_user_matrix] 
                         for user_vector_i in interst_user_matrix]

def most_similar_interests_to(interest_id):
    similarities = interest_similarities[interest_id]
    pairs = [(unique_interests[other_interest_id], similarity)
    for other_interest_id, similarity in enumerate(similarities)
        if interest_id != other_interest_id and similarity > 0]
    return sorted(pairs,
                key=lambda x: x[1],
                reverse=True)


In [14]:
def item_based_suggestions(user_id, include_current_interests=False):
    suggestions = defaultdict(float)
    user_interest_vector = user_interest_matrix[user_id]
    for interest_id, is_interested in enumerate(user_interest_vector):
        if is_interested == 1:
            similar_interests = most_similar_interests_to(interest_id)
            for interst, similarity in similar_interests:
                suggestions[interst] += similarity
    suggestions = sorted(suggestions.items(),
                         key=lambda x: x[1],
                         reverse=True)
    if include_current_interests:
        return suggestions
    else:
        return [(suggestion, weight)
                for suggestion, weight in suggestions
                if suggestion not in users_interests[user_id]]
        

item_based_suggestions(0)

[('MapReduce', 1.861807319565799),
 ('MongoDB', 1.3164965809277263),
 ('Postgres', 1.3164965809277263),
 ('NoSQL', 1.2844570503761732),
 ('MySQL', 0.5773502691896258),
 ('databases', 0.5773502691896258),
 ('Haskell', 0.5773502691896258),
 ('programming languages', 0.5773502691896258),
 ('artificial intelligence', 0.4082482904638631),
 ('deep learning', 0.4082482904638631),
 ('neural networks', 0.4082482904638631),
 ('C++', 0.4082482904638631),
 ('Python', 0.2886751345948129),
 ('R', 0.2886751345948129)]